<h2>Part 3: Explore, Cluster, Map</h2>

This will create a dataframe using data imported from *https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M* and *http://cocl.us/Geospatial_data*. If you have seen the previous associated notebooks which can be found at *https://github.com/Bricksplat/coursera-ibmdspccapstone/blob/master/SegmentCluster%20Toronto%20Part%201%20of%203.ipynb* and *https://github.com/Bricksplat/coursera-ibmdspccapstone/blob/master/SegmentCluster%20Toronto%20Part%202%20of%203.ipynb*, a lot of this will look very familiar, even this introduction and the line right after!

Much of the code seen in this notebook along with the process used is borrowed from the Coursera lab example. I don't fully understand the logic or have the knowledge of Python needed to write these from scratch just yet.

Start with importing the Python libraries that will be used. Some modules are not immediately available and will also need to be installed before importing can happen.

In [1]:
import pandas as pd
import requests
import json
import sklearn
import numpy as np

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0
import folium

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.



This block of code reads an html link and gets the tables that can be found on that page. It picks the desired table and cleans it by removing rows that do not have assigned values for Borough and filling out values for Neighbourhood where needed. Rows are condensed based on shared postcodes with associated Neighbourhoods combined into one.

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
n_url = requests.get(url)

list_tables = pd.read_html(n_url.text)
postal_table = list_tables[0]

for i in range(len(postal_table)-1):
    if postal_table['Neighbourhood'][i] == 'Not assigned':
        postal_table['Neighbourhood'][i] = postal_table['Borough'][i]

postal_table = postal_table[postal_table.Borough != 'Not assigned']
postal_table_indexed = postal_table.reset_index(drop=True)
postal_table_sorted = postal_table_indexed.sort_values(by=['Postcode'])

for i in range(len(postal_table_sorted)-1):
    if postal_table_sorted['Postcode'][i] == postal_table_sorted['Postcode'][i+1]:
        postal_table_sorted['Neighbourhood'][i+1] = (postal_table_sorted['Neighbourhood'][i] + ', ' + postal_table_sorted['Neighbourhood'][i+1])
        postal_table_sorted.drop([i], inplace=True)

postal_table_combined = postal_table_sorted
postal_table_combined = postal_table_combined.reset_index(drop=True)

coord = pd.read_csv('Geospatial_Coordinates.csv')
coord.rename(columns={"Postal Code":"Postcode"}, inplace=True)

postal_table_coord = postal_table_combined.join(coord.set_index('Postcode'), on='Postcode')
postal_table_coord.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


With our neighbourhood location data ready to go, it's time to begin mapping the neighbourhoods using Folium. First, we'll need to set a starting point for our map so we'll get the coordinates for our desired starting point of Toronto, Ontario, Canada.

In [3]:
address = 'Toronto, ON, Canada'

geolocator = Nominatim(user_agent="explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographic coordinates of Toronto, Ontario, Canada are Latitude: {}, Longitude: {}.'.format(latitude, longitude))

The geographic coordinates of Toronto, Ontario, Canada are Latitude: 43.653963, Longitude: -79.387207.


With our starting coordinates defined, we fill in the parameters for our Folium map of Toronto. The markers represent each of the neighbourhoods along with the accompanying borough and general location in Toronto.

In [4]:
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=12)

for latitude, longitude, borough, neighbourhood in zip(postal_table_coord['Latitude'], postal_table_coord['Longitude'], postal_table_coord['Borough'], postal_table_coord['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.8,
        parse_html=False).add_to(toronto_map)  
    
toronto_map

This map looks pretty nice. It is a bit blue with the markers and the ocean but that's okay. Let's move on to exploring these marked neighbourhoods using the Foursquare API. We start with defining the necessary credentials so we can actually make our calls to the API. We will also set our search radius to 500 meters and limit the returned results to 100. We won't actually reach anywhere close to that number though.

In [5]:
CLIENT_ID = 'QS5G2JB5AQAWPY0ASRDOFWRPKIYAYY2U544P4CZFTHJJINBU'
CLIENT_SECRET = 'S5C2XZ3M2IDHTCMS3ZGUERP33G2PG0PXQKO04CWL55DXQXSG'
VERSION = '20180605'

In [6]:
radius = 500 # radius is in meters
LIMIT = 100

We need to get the nearby venues using calls to the Foursquare API and the parameters that have just been defined. The results returned from this call will come in the form of a json file containing a lot of information about each venue. Not all of this information is needed so the function will also have a pick and choose component to it.

In [7]:
venues_list = []

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Let's run *postal_table_coord* through the function and see what we get.

In [8]:
toronto_venues = getNearbyVenues(names=postal_table_coord['Neighbourhood'],
                                   latitudes=postal_table_coord['Latitude'],
                                   longitudes=postal_table_coord['Longitude']
                                  )

venues_list


Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

[[('Rouge, Malvern',
   43.806686299999996,
   -79.19435340000001,
   "Wendy's",
   43.80744841934756,
   -79.19905558052072,
   'Fast Food Restaurant'),
  ('Rouge, Malvern',
   43.806686299999996,
   -79.19435340000001,
   'Interprovincial Group',
   43.8056297,
   -79.2003784,
   'Print Shop')],
 [('Highland Creek, Rouge Hill, Port Union',
   43.7845351,
   -79.16049709999999,
   'Chris Effects Painting',
   43.784343,
   -79.163742,
   'Construction & Landscaping'),
  ('Highland Creek, Rouge Hill, Port Union',
   43.7845351,
   -79.16049709999999,
   'Royal Canadian Legion',
   43.78253332838298,
   -79.16308473261682,
   'Bar'),
  ('Highland Creek, Rouge Hill, Port Union',
   43.7845351,
   -79.16049709999999,
   'Affordable Toronto Movers',
   43.787918740889744,
   -79.16297673924419,
   'Moving Target')],
 [('Guildwood, Morningside, West Hill',
   43.7635726,
   -79.1887115,
   'Swiss Chalet Rotisserie & Grill',
   43.76769708292701,
   -79.1899135003439,
   'Pizza Place'),
  ('

That looks like a lot of places and a shape function confirms that it is a lot of places with 2232 venue entries found.

In [9]:
print(toronto_venues.shape)

(2241, 7)


A small preview shows some of the variety in venues that can be expected with each of the 5 preview venues being in different categories.

In [10]:
toronto_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge, Malvern",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
3,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
4,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target


We have the number of venues but with 2232 entries to look through, it's not readily apparent how many venues there are in each neighbourhood. Group by solves that problem and also shows a few interesting things. Take the first entry of Adelaide, King, Richmond with its 100 venues. If we wanted to, we could increase the limit we set earlier at 100 to better find out how many venues there are, but it's really not the main goal here.

In [11]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,5,5,5,5,5,5
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",3,3,3,3,3,3
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",8,8,8,8,8,8
"Alderwood, Long Branch",9,9,9,9,9,9
"Bathurst Manor, Downsview North, Wilson Heights",19,19,19,19,19,19
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",23,23,23,23,23,23
Berczy Park,55,55,55,55,55,55


Earlier we used the head function to see the first 5 entries and saw that each one of those 5 entries was in a different venue category. How many of these categories are there though?

In [12]:
print('There are {} venue categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 272 venue categories.


We know the number of venues in each neighbourhood but we don't yet know how many of each venue category are in each neighbourhood. One-hot encoding will help us figure that out.

In [13]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
toronto_group = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_group

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,...,0.0,0.020000,0.00,0.000000,0.000000,0.000000,0.010000,0.000000,0.0,0.000000
1,Agincourt,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.00,0.125000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
4,"Alderwood, Long Branch",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
5,"Bathurst Manor, Downsview North, Wilson Heights",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
6,Bayview Village,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
7,"Bedford Park, Lawrence Manor East",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.043478,...,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
8,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.018182,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
9,"Birch Cliff, Cliffside West",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000


We still  have a huge amount of data that is not easily displayed. The first entry of Adelaide, King, Richmond has at least 100 venues. However, looking above shows a lot of zeroes with two exceptions in vegetarian/vegan restaurant and wine bar. Most of the interesting stuff is hidden in the dots. The following will pull out some of the top venue categories for each neighbourhood.

In [15]:
num_top_venues = 5

for hood in toronto_group['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_group[toronto_group['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
             venue  freq
0      Coffee Shop  0.07
1             Café  0.05
2       Steakhouse  0.04
3  Thai Restaurant  0.04
4           Bakery  0.03


----Agincourt----
                       venue  freq
0                     Lounge   0.2
1             Breakfast Spot   0.2
2  Latin American Restaurant   0.2
3               Skating Rink   0.2
4             Clothing Store   0.2


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                 venue  freq
0                 Park  0.33
1           Playground  0.33
2  Arts & Crafts Store  0.33
3    Accessories Store  0.00
4   Mexican Restaurant  0.00


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                  venue  freq
0   Fried Chicken Joint  0.12
1         Grocery Store  0.12
2              Pharmacy  0.12
3  Fast Food Restaurant  0.12
4           Pizza Place  0.12


----Alderwood, Long Branch----
          ven

           venue  freq
0  Grocery Store  0.33
1           Park  0.17
2           Bank  0.17
3          Hotel  0.17
4  Shopping Mall  0.17


----Downsview, North Park, Upwood Park----
                        venue  freq
0                      Bakery  0.33
1                        Park  0.33
2  Construction & Landscaping  0.33
3           Accessories Store  0.00
4               Metro Station  0.00


----East Birchmount Park, Ionview, Kennedy Park----
               venue  freq
0     Discount Store  0.25
1   Department Store  0.12
2  Convenience Store  0.12
3         Hobby Shop  0.12
4        Bus Station  0.12


----East Toronto----
                venue  freq
0                Park  0.50
1         Coffee Shop  0.25
2   Convenience Store  0.25
3   Accessories Store  0.00
4  Mexican Restaurant  0.00


----Emery, Humberlea----
                             venue  freq
0           Furniture / Home Store   0.5
1                   Baseball Field   0.5
2               Mexican Restaurant   0.0
3  

              venue  freq
0       Coffee Shop  0.21
1               Gym  0.05
2  Sushi Restaurant  0.05
3             Diner  0.05
4              Park  0.05


----Rosedale----
                        venue  freq
0                        Park  0.50
1                       Trail  0.25
2                  Playground  0.25
3           Accessories Store  0.00
4  Modern European Restaurant  0.00


----Roselawn----
                             venue  freq
0          Health & Beauty Service   0.5
1                           Garden   0.5
2                Accessories Store   0.0
3               Mexican Restaurant   0.0
4  Molecular Gastronomy Restaurant   0.0


----Rouge, Malvern----
                        venue  freq
0        Fast Food Restaurant   0.5
1                  Print Shop   0.5
2               Metro Station   0.0
3  Modern European Restaurant   0.0
4           Mobile Phone Shop   0.0


----Runnymede, Swansea----
                       venue  freq
0                       Café  0.08
1   

In [16]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [17]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_group['Neighbourhood']

for ind in np.arange(toronto_group.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_group.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Thai Restaurant,Steakhouse,Bar,Bakery,Restaurant,Salad Place,Sushi Restaurant,Asian Restaurant
1,Agincourt,Latin American Restaurant,Skating Rink,Lounge,Clothing Store,Breakfast Spot,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Arts & Crafts Store,Playground,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Fried Chicken Joint,Fast Food Restaurant,Pharmacy,Video Store,Beer Store,Pizza Place,Sandwich Place,Dog Run,Dessert Shop
4,"Alderwood, Long Branch",Pizza Place,Skating Rink,Coffee Shop,Gym,Pub,Pharmacy,Dance Studio,Sandwich Place,Discount Store,Department Store


We will cluster the neighbourhoods into 5 clusters and combine it so everything is viewable through a single dataframe.

In [18]:
kclusters = 5

toronto_group_clustering = toronto_group.drop('Neighbourhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_group_clustering)

kmeans.labels_[0:10]

array([4, 4, 2, 1, 1, 1, 4, 1, 4, 4])

In [19]:
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merge = postal_table_coord

toronto_merge = toronto_merge.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
toronto_merge.dropna(inplace=True)
toronto_merge['Cluster Labels'] = toronto_merge['Cluster Labels'].astype('int64')

toronto_merge.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,1,Fast Food Restaurant,Print Shop,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,4,Moving Target,Construction & Landscaping,Bar,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,4,Electronics Store,Medical Center,Mexican Restaurant,Spa,Breakfast Spot,Pizza Place,Rental Car Location,Intersection,Dim Sum Restaurant,Diner
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1,Coffee Shop,Pharmacy,Korean Restaurant,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,4,Gas Station,Bakery,Fried Chicken Joint,Caribbean Restaurant,Athletics & Sports,Thai Restaurant,Bank,Hakka Restaurant,Dumpling Restaurant,Drugstore


Everything's ready for another Folium map! This time it's not so blue which could mean one of many things depending on your view of blue. We could make all of the markers blue again but then we wouldn't be able to easily distinguish which of these neighbourhoods are in which cluster.

In [20]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merge['Latitude'], toronto_merge['Longitude'], toronto_merge['Neighbourhood'], toronto_merge['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

With the rest of the code below, we can see what is in each of the clusters that we created.

In [21]:
toronto_merge.loc[toronto_merge['Cluster Labels'] == 0, toronto_merge.columns[[2] + list(range(5, toronto_merge.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,"Silver Hills, York Mills",0,Cafeteria,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore,College Rec Center


In [22]:
toronto_merge.loc[toronto_merge['Cluster Labels'] == 1, toronto_merge.columns[[2] + list(range(5, toronto_merge.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Rouge, Malvern",1,Fast Food Restaurant,Print Shop,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
3,Woburn,1,Coffee Shop,Pharmacy,Korean Restaurant,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
8,"Cliffcrest, Cliffside, Scarborough Village West",1,American Restaurant,Motel,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Yoga Studio
13,"Clarks Corners, Sullivan, Tam O'Shanter",1,Pizza Place,Shopping Mall,Fast Food Restaurant,Fried Chicken Joint,Thai Restaurant,Italian Restaurant,Convenience Store,Chinese Restaurant,Pharmacy,Noodle House
15,L'Amoreaux West,1,Fast Food Restaurant,Chinese Restaurant,Grocery Store,Thrift / Vintage Store,Gym,Pharmacy,Pizza Place,Sandwich Place,Bubble Tea Shop,Breakfast Spot
17,Hillcrest Village,1,Mediterranean Restaurant,Golf Course,Fast Food Restaurant,Athletics & Sports,Dog Run,Pool,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store
24,Willowdale West,1,Butcher,Coffee Shop,Pharmacy,Pizza Place,Discount Store,Doner Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
28,"Bathurst Manor, Downsview North, Wilson Heights",1,Coffee Shop,Shopping Mall,Pizza Place,Supermarket,Deli / Bodega,Sushi Restaurant,Middle Eastern Restaurant,Restaurant,Fast Food Restaurant,Diner
33,Downsview Northwest,1,Grocery Store,Liquor Store,Gym / Fitness Center,Athletics & Sports,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
35,"Woodbine Gardens, Parkview Hill",1,Fast Food Restaurant,Pizza Place,Athletics & Sports,Intersection,Bus Line,Gastropub,Bank,Pharmacy,Gym / Fitness Center,Dumpling Restaurant


In [23]:
toronto_merge.loc[toronto_merge['Cluster Labels'] == 2, toronto_merge.columns[[2] + list(range(5, toronto_merge.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,"Agincourt North, L'Amoreaux East, Milliken, St...",2,Park,Arts & Crafts Store,Playground,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
21,"Newtonbrook, Willowdale",2,Park,Piano Bar,Doner Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
23,York Mills West,2,Park,Bank,Convenience Store,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Department Store,Donut Shop
25,Parkwoods,2,Park,Food & Drink Shop,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
30,"CFB Toronto, Downsview East",2,Park,Airport,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
40,East Toronto,2,Park,Coffee Shop,Convenience Store,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Deli / Bodega
50,Rosedale,2,Park,Trail,Playground,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
74,Caledonia-Fairbanks,2,Park,Fast Food Restaurant,Market,Women's Store,Gift Shop,German Restaurant,Electronics Store,Eastern European Restaurant,Golf Course,Dumpling Restaurant
79,"Downsview, North Park, Upwood Park",2,Park,Bakery,Construction & Landscaping,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
98,Weston,2,Convenience Store,Park,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Yoga Studio


In [24]:
toronto_merge.loc[toronto_merge['Cluster Labels'] == 3, toronto_merge.columns[[2] + list(range(5, toronto_merge.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough Village,3,Playground,Yoga Studio,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore


In [25]:
toronto_merge.loc[toronto_merge['Cluster Labels'] == 4, toronto_merge.columns[[2] + list(range(5, toronto_merge.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Highland Creek, Rouge Hill, Port Union",4,Moving Target,Construction & Landscaping,Bar,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
2,"Guildwood, Morningside, West Hill",4,Electronics Store,Medical Center,Mexican Restaurant,Spa,Breakfast Spot,Pizza Place,Rental Car Location,Intersection,Dim Sum Restaurant,Diner
4,Cedarbrae,4,Gas Station,Bakery,Fried Chicken Joint,Caribbean Restaurant,Athletics & Sports,Thai Restaurant,Bank,Hakka Restaurant,Dumpling Restaurant,Drugstore
6,"East Birchmount Park, Ionview, Kennedy Park",4,Discount Store,Hobby Shop,Convenience Store,Bus Station,Chinese Restaurant,Department Store,Coffee Shop,Eastern European Restaurant,Electronics Store,Dumpling Restaurant
7,"Clairlea, Golden Mile, Oakridge",4,Bakery,Bus Line,Soccer Field,Metro Station,Bus Station,Fast Food Restaurant,Intersection,Park,Gastropub,Deli / Bodega
9,"Birch Cliff, Cliffside West",4,Café,College Stadium,Skating Rink,General Entertainment,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
10,"Dorset Park, Scarborough Town Centre, Wexford ...",4,Indian Restaurant,Thrift / Vintage Store,Pet Store,Vietnamese Restaurant,Furniture / Home Store,Chinese Restaurant,Yoga Studio,Department Store,Dessert Shop,Dim Sum Restaurant
11,"Maryvale, Wexford",4,Sandwich Place,Breakfast Spot,Middle Eastern Restaurant,Bakery,Auto Garage,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dim Sum Restaurant
12,Agincourt,4,Latin American Restaurant,Skating Rink,Lounge,Clothing Store,Breakfast Spot,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
18,"Fairview, Henry Farm, Oriole",4,Clothing Store,Coffee Shop,Fast Food Restaurant,Women's Store,Tea Room,Bakery,Food Court,Japanese Restaurant,Bus Station,Shopping Mall
